# Dynamic Schema in Collections

Earlier, we had to declare the field schema of the Milvus collection, and the data inserted into the collection should strictly match the schema.

But Milvus has added support for dynamic schema, which enables us to insert entities with new fields into the collection, but without modifying the schema. This gives us a lot of flexibility in terms of new or undefined fields in the collection schema.

In [1]:
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType, connections, utility
import pandas as pd
import random
import json
import numpy as np

In [2]:
connections.connect(
  alias="default",
  host='localhost',
  port='19530'
)
utility.drop_collection("dynamic_schema_example")

In [3]:
## Field Schema

name = FieldSchema(
  name="name",
  dtype=DataType.VARCHAR,
  max_length=200,
)
id = FieldSchema(
  name="id",
  dtype=DataType.INT64,
  is_primary=True,
)
count = FieldSchema(
  name="count",
  dtype=DataType.INT64,
)
vec = FieldSchema(
  name="vector",
  dtype=DataType.FLOAT_VECTOR,
  dim=64
)


To enable dynamic schema in a milvus collection, we have to set the parameter called enable dynamic field as true while creating the collection.

In [4]:
# Collection schema

collection_schema = CollectionSchema(
  fields=[name, id, count, vec],
  description="Dynamic Schema Example",
    enable_dynamic_field=True
)

In [5]:
# Create collection

collection = Collection(
    name="dynamic_schema_example",
    schema=collection_schema,
    using='default')

utility.list_collections()

['Album1', 'dynamic_schema_example']

In [6]:
index_params = {
  "metric_type":"L2",
  "index_type":"IVF_SQ8",
  "params":{"nlist":64},
  "index_name": "vector_index"
}

collection.create_index(
  field_name="vector", 
  index_params=index_params
)

collection.load()

If you look at this closely, the inserted entities has a field called name, id, count and a field called vector and these four fields are already defined while creating the collection.

But along with this we have a field called rand and this field is not declared in the field scheme of the collection.

But since we have enabled the dynamic schema for this collection, we should be able to insert the data with new fields into the collection.

In [7]:
# Data to be inserted

data = [ {'name': 'a', 'id': 1, 'count': 10, 'vector': [random.uniform(0, 64) for x in range(64)], 'rand':1},
        {'name': 'b', 'id': 2, 'count': 10, 'vector': [random.uniform(0, 64) for x in range(64)], 'rand':2},
        {'name': 'c', 'id': 3, 'count': 10, 'vector': [random.uniform(0, 64) for x in range(64)], 'rand':3},
        {'name': 'd', 'id': 4, 'count': 10, 'vector': [random.uniform(0, 64) for x in range(64)], 'rand':4}]

So we are able to insert the data with new fields into this collection.

In [8]:
# Insert the data in to the collection
collection.insert(data)


(insert count: 4, delete count: 0, upsert count: 0, timestamp: 450512990264623109, success count: 4, err count: 0)

In [10]:
utility.list_collections()

['Album1', 'dynamic_schema_example']